# Import Libraries

In [1]:
import random as rn
from keras import backend as K
import tensorflow as tf
import numpy as np
import os
import datetime
import os
import cv2
from keras.models import Sequential
from keras.layers import LSTM,BatchNormalization, Flatten
from keras.layers.core import Activation, Dropout, Dense
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from keras.optimizers import Adam

Using TensorFlow backend.


### CSV and Image path

In [0]:
ROOT_DIR='./drive/My Drive/Assignment'
ROOT_DIR_TRAIN=ROOT_DIR +'/train'
ROOT_DIR_TEST=ROOT_DIR +'/val'
train_file = 'train.csv'
test_file = 'val.csv'

#### giving the random seed to 30  so that random number will be less than 30

# Now we have to catogorize the images folder 
Image data augmentation is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset.

Training deep learning neural network models on more data can result in more skillful models, and the augmentation techniques can create variations of the images that can improve the ability of the fit models to generalize what they have learned to new images.

https://towardsdatascience.com/image-augmentation-for-deep-learning-histogram-equalization-a71387f609b2

https://towardsdatascience.com/classify-butterfly-images-with-deep-learning-in-keras-b3101fe0f98

image thresholding: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html

In [0]:
class ImageDataGenerator:
  def __init__(self):    
    self.img_index = [x for x in range(0, 20)] 
    self.batch_size = 10

    self.frame = 20
    self.rows = 120
    self.cols = 120  
    self.channel = 3

    # we have to yield for each batch.  For saving resulting creating this data structure.  
    self.batch_data = np.zeros((self.batch_size,self.frame,self.rows,self.cols,self.channel))
    self.batch_labels = np.zeros((self.batch_size,5))

    self.data_augmet, self.label_augmet =  self.batch_data,self.batch_labels
    self.data_flip,self.label_flip = self.batch_data,self.batch_labels

  #  For shuffle  the images and train data  
  def imgagBatchProcessing(self, train_folders, path):
     first = True     
     while first:
       suffled_folder_list = np.random.permutation(train_folders)
       num_batches = len(train_folders)//self.batch_size
       for batch in range(num_batches):
         # Yield here so that the control will come after next run
         yield self.imageGenerator(batch,path,suffled_folder_list)

         # for printing first images un-commeNt here

       #first = False
       #  if first:
       #    self.imageGenerator(batch,first)
       #    first = False
       #  else: 
       #    self.imageGenerator(batch, False)

     # we iterate over the number of batches
     # you yield the batch_data and the batch_labels, remember what does yield do            

  def imageGenerator(self,batch,path,suffled_folder_list):
    try:          
      for folder in range(self.batch_size): # iterate over the batch_size 
        imgs = sorted(os.listdir(path+'/'+ suffled_folder_list[folder + (batch*self.batch_size)].split(';')[0]))
        dx, dy = np.random.randint(-1.7, 1.8, 2)
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        
        for index, item in enumerate(self.img_index):      
          image = cv2.imread(path+'/'+ suffled_folder_list[folder + (batch*self.batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          # resizing the image
          resized_image = self.crop_image(image)

          # Normalizing the image 
          normalizeImage0 = self.normalizeImage(resized_image[:,:,0])
          normalizeImage1 = self.normalizeImage(resized_image[:,:,1])
          normalizeImage2 = self.normalizeImage(resized_image[:,:,2])
          
          x = resized_image.shape[0]
          y = resized_image.shape[1]
          # if first:
          #   self.printImage(image)
          #   self.printImage(resized_image)
          #   self.printImage(normalizeImage0)
          #   first = False

          self.batch_data[folder,index,:,:,0] = normalizeImage0
          self.batch_data[folder,index,:,:,1] = normalizeImage1
          self.batch_data[folder,index,:,:,2] = normalizeImage2

          # warpAffine will transfer the source image to matrix
          # https://docs.opencv.org/3.4/d4/d61/tutorial_warp_affine.html
          self.data_augmet[folder, index] = cv2.warpAffine(resized_image, M ,(x,y))

      self.batch_labels[folder, int(suffled_folder_list[folder + (batch*self.batch_size)].strip().split(';')[2])] = 1
      self.label_augmet[folder, int(suffled_folder_list[folder + (batch*self.batch_size)].strip().split(';')[2])] = 1
      #self.data_flip[folder, index] = np.flip(resized_image, 1) 

      data_final = np.append(self.batch_data, self.data_augmet)
      data_label_final = np.append(self.batch_labels, self.label_augmet)

      prin(len(data_final))

      return data_final, data_label_final

    except Exception as e:
      #print(e)
      pass

  def crop_image(self, img):
    from skimage.transform import resize
    if img.shape[0] != img.shape[1]:
        img=img[0:120,10:150]
    resized_image = resize(img, (self.rows,self.cols))
    return resized_image

  def normalizeImage(self,img):
    #using percentile for normalization for images, as min-max is giving better results.
    # normalized_image= img - np.percentile(img,15)/ np.percentile(img,85) - np.percentile(img,15) 
    normalized_image= (img - np.min(img))/(np.max(img)- np.min(img))
    return normalized_image
     
  def printImage(self, image):
    %pylab inline
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    imgplot = plt.imshow(image)
    plt.show()      


In [0]:
imageTrainer  =  ImageDataGenerator()

train_folders = np.random.permutation(open(ROOT_DIR +'/'+ train_file).readlines())
test_folders = np.random.permutation(open(ROOT_DIR +'/'+ test_file).readlines())

train_generator  = imageTrainer.imgagBatchProcessing(train_folders, ROOT_DIR_TRAIN)
test_generator  = imageTrainer.imgagBatchProcessing(test_folders, ROOT_DIR_TEST)


## Model Using CONV3d


In [0]:
#model = Sequential()
#model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
#model.add(Dense(1))
#model.compile(optimizer='adam', loss='mse')

# from keras.layers.embeddings import Embedding

# embedding_vecor_length = 32
# model = Sequential()
# model.add(Embedding(5000, embedding_vecor_length, input_length=500))
# model.add(LSTM(100))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

nb_featuremap = [8,16,32,64]
nb_dense = [128,64,5]
nb_classes = 5
frames = 20
rows = 120
cols = 120  
channel = 3

from keras.layers.convolutional import Conv3D,Conv2D, MaxPooling3D,MaxPooling2D
input_shape=(frames,rows,cols,channel)

model3d = Sequential()
model3d.add(Conv3D(nb_featuremap[0], 
                 kernel_size=(5,5,5),
                 input_shape=input_shape,
                 padding='same', name="conv1"))
model3d.add(Activation('relu'))
model3d.add(Conv3D(nb_featuremap[1], 
                 kernel_size=(3,3,3),
                 padding='same',name="conv2"))
model3d.add(Activation('relu'))
model3d.add(MaxPooling3D(pool_size=(2,2,2)))
model3d.add(Conv3D(nb_featuremap[2], 
                 kernel_size=(1,3,3), 
                 padding='same',name="conv3"))
model3d.add(Activation('relu'))
model3d.add(MaxPooling3D(pool_size=(2,2,2)))
model3d.add(BatchNormalization())
model3d.add(Dropout(0.25))
model3d.add(MaxPooling3D(pool_size=(2,2,2)))
model3d.add(Flatten())
model3d.add(Dense(nb_dense[0], activation='relu'))
model3d.add(Dropout(0.25))
model3d.add(Dense(nb_dense[1], activation='relu'))
#softmax layer
model3d.add(Dense(nb_dense[2], activation='softmax'))
optimiser = Adam(0.001)
model3d.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [0]:

batch_size = 10
num_epochs = 30 # the number of epochs

num_train_sequences = len(train_folders)
num_val_sequences = len(test_folders)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1


In [7]:
# model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
#                     validation_data=test_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0,use_multiprocessing=True)

model3d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, 
                      verbose=1,validation_data=test_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0,use_multiprocessing=True)

Epoch 1/30


Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 650, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "<ipython-input-3-5db6b31faad8>", line 26, in imgagBatchProcessing
    yield self.imageGenerator(batch,path,suffled_folder_list)
  File "<ipython-input-3-5db6b31faad8>", line 48, in imageGenerator
    image = cv2.imread(path+'/'+ suffled_folder_list[folder + (batch*self.batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
KeyboardInterrupt


KeyboardInterrupt: ignored

Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 650, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "<ipython-input-3-5db6b31faad8>", line 26, in imgagBatchProcessing
    yield self.imageGenerator(batch,path,suffled_folder_list)
  File "<ipython-input-3-5db6b31faad8>", line 48, in imageGenerator
    image = cv2.imread(path+'/'+ suffled_folder_list[folder + (batch*self.batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
KeyboardInterrupt


In [0]:
model3d.save('gesture_lstm.h5')

In [10]:
from keras.models import load_model

model = load_model('gesture_lstm.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 20, 120, 120, 8)   3008      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 120, 120, 8)   0         
_________________________________________________________________
conv2 (Conv3D)               (None, 20, 120, 120, 16)  3472      
_________________________________________________________________
activation_2 (Activation)    (None, 20, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 60, 60, 16)    0         
_________________________________________________________________
conv3 (Conv3D)               (None, 10, 60, 60, 32)    4640      
_________________________________________________________________
activation_3 (Activation)    (None, 10, 60, 60, 32)   

In [0]:
# fuctions returns all files of given format within given directory
def createImageFileList(dir, format='.png'):
    fileList = []
    for root, dirs, files in os.walk(dir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)   
    return fileList

In [0]:
_xTest = []
_yTest = []
num_images_test = 0
print(test_folders.size)

for folder in range(test_folders.size):
  try:
    imagepaths = [] # Image paths   
    fn = test_folders[folder].strip().split(';')
    path = ROOT_DIR_TEST+'/'+ fn[0]
    class_index = int(fn[2])
    
    imagepaths = createImageFileList(path)
    # Load each image into X[] & y[]
    for image in imagepaths:
        img = cv2.imread(image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (120,120))
        # Build training example, label
        num_images_test = num_images_test + 1
        _xTest.append(img)
        _yTest.append(class_index)
  except:
   continue

101


Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/us

In [0]:
print("Num Images: ", num_images_test)
_xTest = np.array(_xTest, dtype='uint8')
_xTest = _xTest.reshape(num_images_test, 240, 240, 1)
_yTest = np.array(_yTest)

In [0]:
preds = model.predict(_xTest)

In [0]:
np.unique(pred_classes)

In [0]:
import seaborn as sns
# Confusion metrics  
# the  below diagram represents  as: 
#############
#  TP # FN #
############
#  FP # TN #
###########
# where T True,  F false  N -ve and  P +ve  
#  Precision will be  ( TP / ( TP+FP ) )
#  Recall  will be  ( TP / ( TP + FN ) )
labels = [1,0]
cm = confusion_matrix(_yTest, pred_classes, labels)
sns.heatmap(cm, fmt="0.0f", xticklabels=['Yes', 'No'], yticklabels=['Yes', 'No'], annot=True)
plt.title("Confusion MatrixR")
plt.xlabel('predicted')
plt.ylabel('actual') 

In [0]:
print(accuracy_score(_yTest, pred_classes))

In [0]:

# model3d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, 
#                       verbose=1,validation_data=test_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0,use_multiprocessing=True)

In [0]:

# y_pred = model.predict_classes(Xv)